# Assignment 2
## Assignment submission header
### Submission preparation instructions
Please provide the names and Drexel email addresses of all group members. If applicable, provide notes on any tutoring support received toward the completion of this submission.

### Assignment submission group
- Group member 1: Manav (md3834@drexel.edu)


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Part 1 (25 points)

In this section, convert the comma-delimiated format provided for schedules (as shown [here](http://www3.septa.org/ccstations/me/sched_data.csv)) into a functioning schedule (as shown [here](http://www3.septa.org/ccstations/me/)). Note: Additional documentation may be found [here](http://www3.septa.org/).

__1a.__ (3 points) Update the station code to get the schedule for Suburban Station using SEPTA's API (as shown in the example in __Section 3.1.3__ of the lecture notes). Print the time at which the request was made. Print and examine the second item in `schedule`.

In [40]:
import re, requests, csv
from pprint import pprint
from datetime import datetime as dt

#---Your code starts here---
station_code = '30th'
#---Your code stops here---
schedule_url = "".join(["http://www3.septa.org/ccstations/", station_code, "/sched_data.csv"])
response = requests.get(schedule_url)
schedule = list(csv.reader(response.text.strip().split("\n")))

print("Access time:", dt.now())
#---Your code starts here---
pprint(schedule[1])
#---Your code stops here---

Access time: 2024-05-15 00:48:04.984975
['R4S=08:59',
 'Airport',
 '6',
 ' 3 LATE',
 'LOCAL                    ',
 '8467  ',
 '<_NEXT_MSG>09:29',
 'Airport',
 '6',
 'ON TIME',
 'LOCAL                    ',
 '469   ',
 '<_NEXT_MSG>09:59',
 'Airport',
 '6',
 'ON TIME',
 'LOCAL                    ',
 '8471  ',
 '<_NEXT_MSG>10:29',
 'Airport',
 '6',
 'ON TIME',
 'LOCAL                    ',
 '473   ',
 '']


__1b.__ (8 points) Create a list called `trains` that extracts three pieces of information from `schedule` for each train: (1) scheduled arrival time, (2) destination, and (3) status. Store this information as a list of lists. Print the first five lists in `trains`. (Note: There may be a variable number of trains for each item in `schedule`, but each train has a fixed number of attributes.)

In [41]:
trains = []

#---Your code starts here---
for train_info in schedule:
    # Check if the train_info list has at least four elements
    if len(train_info) >= 4:
        # Extract scheduled arrival time, destination, and status
        arrival_time = train_info[0].split('=')[1]  # Split and take the second part
        destination = train_info[1]
        status = train_info[3]

        # Store the extracted information as a list
        train = [arrival_time, destination, status]

        # Append the list to the 'trains' list
        trains.append(train)
#---Your code stops here---

trains[:5]

[['08:59', 'Airport', ' 3 LATE'],
 ['09:00', 'Warminster', 'ON TIME'],
 ['10:39', 'Wilmington', 'ON TIME'],
 ['09:15', 'West Trenton', 'ON TIME'],
 ['09:03', 'Wawa', 'ON TIME']]

__1c.__ (5 points) The arrival times are listed in 12-hour time, but they lack AM/PM information. Update `trains` by using the current system time and the fact that the schedule information contains only trains arriving in the next few hours to fix the problem (the updated arrival times should have the following format: mm/dd/yyyy hh:mm[AM/PM]). Print the first five items in the updated list. (Note: Make sure your method can handle trains schedule to arrive after midnight.)

In [42]:
from pytz import timezone

access_time = dt.now(timezone('US/Eastern'))
access_hour = access_time.hour if access_time.hour else 12
access_date = access_time.strftime("%m/%d/%Y")

if access_hour >= 12:
  access_hour -= 12
  current_am_or_pm = ["PM", "AM"]
else:
  current_am_or_pm = ["AM", "PM"]

for train in trains:
    arrival_time_str = train[0]
    arrival_hour, arrival_minute = map(int, arrival_time_str.split(":"))

    # Determine AM/PM for the arrival time
    if access_hour <= arrival_hour < access_hour + 12:
        am_pm = current_am_or_pm[0]
    else:
        am_pm = current_am_or_pm[1]

    formatted_arrival_time = f"{access_date} {arrival_hour:02}:{arrival_minute:02}{am_pm}"
    train[0] = formatted_arrival_time  # Update the arrival time in the train list

  #---Your code stops here---


trains[:5]

[['05/14/2024 08:59PM', 'Airport', ' 3 LATE'],
 ['05/14/2024 09:00PM', 'Warminster', 'ON TIME'],
 ['05/14/2024 10:39PM', 'Wilmington', 'ON TIME'],
 ['05/14/2024 09:15PM', 'West Trenton', 'ON TIME'],
 ['05/14/2024 09:03PM', 'Wawa', 'ON TIME']]

__1d.__ (4 points) Complete the `parse_time` function below, which takes a schedule such as that stored in `trains` as input and parses the arrival time data using `dateutil.parser`. The function should return a list that includes three pieces of information: (1) parsed arrival time, (2) destination, and (3) status. Prior to returning, sort the data in ascending order of arrival time. Print the first five items in the parsed data.

In [43]:
from dateutil import parser as dateparser

def parse_times(trains):

    datetime_parsed_trains = []

    #---Your code starts here---
    for train in trains:
        arrival_time_str = train[0]
        destination = train[1]
        status = train[2]

        # Parse the arrival time using dateutil.parser
        parsed_arrival_time = dateparser.parse(arrival_time_str)

        # Append the parsed information as a tuple to the list
        datetime_parsed_trains.append((parsed_arrival_time, destination, status))
    #---Your code stops here---

    return sorted(datetime_parsed_trains, key = lambda x: x[0])

In [44]:
parsed_trains = parse_times(trains)
parsed_trains[:5]

[(datetime.datetime(2024, 5, 14, 20, 44), 'Trenton', ' 5 LATE'),
 (datetime.datetime(2024, 5, 14, 20, 44), 'Trenton', ' 5 LATE'),
 (datetime.datetime(2024, 5, 14, 20, 52), 'Norristown', ' 1 LATE'),
 (datetime.datetime(2024, 5, 14, 20, 52), 'Chestnut H West', 'ON TIME'),
 (datetime.datetime(2024, 5, 14, 20, 52), 'Norristown', ' 1 LATE')]

__1e.__ (5 points) Complete the `save_schedule` function to create hourly log files with train information. File names should be formatted as `"data/trains/%Y-%m%-d%-H%.txt"`. Files should contain the updated arrival time, destination, and status of each train scheduled to arrive within the hour, with one train per line. Apply `save_schedule` to the parsed version of the train schedule, and print the contents of the `"data/trains/"` directory.

In [45]:
import os

def save_schedule(datetime_parsed_trains_24_hour):
    os.system("mkdir -p data/trains/")

    #---Your code starts here---
    # Create a dictionary to hold the trains for each hour
    hourly_trains = {}

    for train in datetime_parsed_trains_24_hour:
        arrival_time, destination, status = train
        hour_str = arrival_time.strftime("%Y-%m-%d-%H")

        if hour_str not in hourly_trains:
            hourly_trains[hour_str] = []

        # Format the train information
        train_info = "{} {} {}".format(arrival_time.strftime('%m/%d/%Y %I:%M%p'), destination, status)
        hourly_trains[hour_str].append(train_info)

    # Write the train information to the respective files
    for hour, trains in hourly_trains.items():
        file_path = "data/trains/{}.txt".format(hour)
        with open(file_path, 'w') as f:
            for train in trains:
                f.write(train + "\n")
    #---Your code stops here---

# Apply save_schedule to the parsed version of the train schedule
save_schedule(parsed_trains)

# Print the contents of the "data/trains/" directory
print([x for x in os.listdir("data/trains/") if re.search(parsed_trains[0][0].strftime("%Y-%m-%d-\d\d.txt"), x)])


['2024-05-14-20.txt', '2024-05-14-21.txt', '2024-05-14-22.txt']


# Part 2 (25 points)
For this section, you will use the Sportradar Soccer v4 API to download and process the results of matches in the 2020-2021 season of the English Premier League. API documentation can be found [here.](https://api.sportradar.com/soccer/trial/v4/openapi/swagger/index.html#/seasons/getSeasons)

__2a.__ (4 points) Sign up for a Sportradar developer account and obtain an API key for the "Soccer v4" endpoints. Once you've registered an app, create a key, and update `soccer_key`. After making the API request below to request summary information from the 2022&ndash;2023 season, print the first item under "summaries" in `season_summary` and explore the data.  

In [33]:
import requests

#---Your code starts here---
soccer_key = "ZMaoP9iwP398Qj6qanm8565thLWHLgBZ4Mia0FLH"
#---Your code stops here---
competition_id = "sr:season:94193"

address = "".join([
    "https://api.sportradar.com/soccer/trial/v4/en/seasons/",
    competition_id,
    "/summaries.json?api_key=",
    soccer_key
])
season_summary = requests.get(address).json()

season_summary["summaries"][0]

{'sport_event': {'id': 'sr:sport_event:34246413',
  'start_time': '2022-07-15T17:00:00+00:00',
  'start_time_confirmed': True,
  'sport_event_context': {'sport': {'id': 'sr:sport:1', 'name': 'Soccer'},
   'category': {'id': 'sr:category:21',
    'name': 'Russia',
    'country_code': 'RUS'},
   'competition': {'id': 'sr:competition:203',
    'name': 'Premier League',
    'gender': 'men'},
   'season': {'id': 'sr:season:94193',
    'name': 'Premier League 22/23',
    'start_date': '2022-07-15',
    'end_date': '2023-06-10',
    'year': '22/23',
    'competition_id': 'sr:competition:203'},
   'stage': {'order': 1,
    'type': 'league',
    'phase': 'regular season',
    'start_date': '2022-07-15',
    'end_date': '2023-06-03',
    'year': '22/23'},
   'round': {'number': 1},
   'groups': [{'id': 'sr:league:66085', 'name': 'Premier League 22/23'}]},
  'coverage': {'type': 'sport_event',
   'sport_event_properties': {'lineups': True,
    'venue': True,
    'extended_player_stats': False,
  

__2b.__ (6 points) Create a function called `get_team_results` that takes `season_summary` as input and construct a dictionary that uses abbreviated team names as keys and collects the following information for each match played by each team (`status != "postponed"`): (1) abbreviation of the opposing team, (2) goals scored, (3) goals conceded, and (4) points earned. (Note: In soccer leagues, winning a match earns 3 points for the winner and 0 points for the loser, while drawing results in 1 point for each team. The goal difference is the difference between goals for and goals against.) Store the results as `team_results`, and print the first five matches played by "SOC".

In [34]:
from collections import defaultdict

def get_team_results(season_summary):
  team_results = defaultdict(list)

  for match in season_summary["summaries"]:
    if match["sport_event_status"]['status'] != 'postponed':

      for team in match["sport_event"]["competitors"]:
          #---Your code starts here---(collect the home and away team abbreviations)
          if team["qualifier"] == "home":
                    home_team = team["abbreviation"]
          elif team["qualifier"] == "away":
            away_team = team["abbreviation"]

            # Collect the home and away scores
      home_score = match["sport_event_status"]["home_score"]
      away_score = match["sport_event_status"]["away_score"]

      # Calculate the home and away points
      if home_score > away_score:
        home_points = 3
        away_points = 0
      elif home_score < away_score:
        home_points = 0
        away_points = 3
      else:
        home_points = 1
        away_points = 1


      team_results[home_team].append({"opponent": away_team,
                                      "scored" : home_score,
                                      "conceded" : away_score,
                                      "points" : home_points})

      team_results[away_team].append({"opponent": home_team,
                                      "scored" : away_score,
                                      "conceded" : home_score,
                                      "points" : away_points})

  return team_results

In [35]:
team_results = get_team_results(season_summary)
team_results["SOC"][:5]

[{'opponent': 'TOR', 'scored': 3, 'conceded': 1, 'points': 3},
 {'opponent': 'CSKA', 'scored': 0, 'conceded': 3, 'points': 0},
 {'opponent': 'AKH', 'scored': 2, 'conceded': 1, 'points': 3},
 {'opponent': 'NNO', 'scored': 2, 'conceded': 1, 'points': 3},
 {'opponent': 'SPA', 'scored': 0, 'conceded': 3, 'points': 0}]

__2c.__ (5 points) Aggregate the results data for each team (key) in `team_results`, and store this information as a list called `standings`. Here, collect the abbreviated team name, the number of games played, the numbers of games won, drawn, and lost, the number of goals scored and conceded, and total points. Print the first five items in `standings`.

In [36]:
standings = []

for team in team_results:
    games_played = 0
    games_won = 0
    games_drawn = 0
    games_lost = 0
    goals_scored = 0
    goals_conceded = 0
    points_earned = 0

    #---Your code starts here---
    for match in team_results[team]:
        games_played += 1
        goals_scored += match["scored"]
        goals_conceded += match["conceded"]
        points_earned += match["points"]

        if match["points"] == 3:
            games_won += 1
        elif match["points"] == 1:
            games_drawn += 1
        else:
            games_lost += 1
    #---Your code stops here---

    standings.append([team, games_played, games_won, games_drawn, games_lost,
                      goals_scored, goals_conceded, points_earned])

# Print the first five items in standings
print(standings[:5])

[['KHI', 13, 2, 3, 8, 13, 23, 9], ['ZEN', 12, 9, 3, 0, 33, 5, 30], ['CSKA', 12, 7, 3, 2, 24, 11, 24], ['URA', 13, 4, 1, 8, 13, 23, 13], ['ORE', 12, 6, 0, 6, 20, 28, 18]]


__2d.__ (5 points) In the English Premier League, standings are calculated based on points earned, but let's see how points earned compares to the number of goals earned. First, sort `standings` by the number of goals scored to see which team scored the most points. Then, sort `standings` by the number of `points` to see who won the tournament. Was the team that scored the most points the same team that won the tournament?

In [37]:
# Sort standings by the number of goals scored
standings_sorted_by_goals = sorted(standings, key=lambda x: x[5], reverse=True)

# Print the team that scored the most goals
print("Most goals:", standings_sorted_by_goals[0])

# Sort standings by the number of points
standings_sorted_by_points = sorted(standings, key=lambda x: x[7], reverse=True)

# Print the team that scored the most points
print("Most points:", standings_sorted_by_points[0])

# Compare the top teams
if standings_sorted_by_goals[0][0] == standings_sorted_by_points[0][0]:
    print("Yes, the team that scored the most goals is the same as the team who scored the most points.")
else:
    print("No, the team that scored the most goals is not the same as the team who scored the most points.")


Most goals: ['ZEN', 12, 9, 3, 0, 33, 5, 30]
Most points: ['ZEN', 12, 9, 3, 0, 33, 5, 30]
Yes, the team that scored the most goals is the same as the team who scored the most points.


__2e.__ (5 points) If two teams have the same number of points, then the team with the higher difference between goals earned and goals conceded is ranked higher. If the differences are also the same, then the team with the higher number of goals scored is ranked higher. To correctly rank the teams, sort `standings` by the three factors described above. Print the sorted list.

In [39]:
# Define the sorting key with the three criteria
def sorting_key(team_stats):
    points = team_stats[7]
    goal_difference = team_stats[5] - team_stats[6]  # goals scored - goals conceded
    goals_scored = team_stats[5]
    return (points, goal_difference, goals_scored)

# Sort standings by points, then goal difference, then goals scored
standings_sorted = sorted(standings, key=sorting_key, reverse=True)

#---Your code stops here---
standings

[['KHI', 13, 2, 3, 8, 13, 23, 9],
 ['ZEN', 12, 9, 3, 0, 33, 5, 30],
 ['CSKA', 12, 7, 3, 2, 24, 11, 24],
 ['URA', 13, 4, 1, 8, 13, 23, 13],
 ['ORE', 12, 6, 0, 6, 20, 28, 18],
 ['KSS', 12, 2, 5, 5, 12, 19, 11],
 ['AKH', 13, 6, 2, 5, 21, 21, 20],
 ['SPA', 12, 8, 1, 3, 29, 14, 25],
 ['TOR', 13, 1, 3, 9, 7, 24, 6],
 ['SOC', 12, 6, 2, 4, 22, 20, 20],
 ['LOK', 13, 3, 3, 7, 16, 26, 12],
 ['NNO', 12, 3, 4, 5, 17, 20, 13],
 ['KRA', 13, 6, 3, 4, 25, 20, 21],
 ['FAK', 13, 1, 8, 4, 16, 23, 11],
 ['DMO', 12, 4, 5, 3, 18, 14, 17],
 ['ROS', 13, 7, 4, 2, 24, 19, 25]]

# Part 3 (25 points)

__3a.__ (3 points) Update `file_path` and `delim` to read the text from `data/tempest.txt` and split its content into scenes. (Note: `delim` should be set to a pattern that matches the scene headers (__Section 4.4.1.2__)). Print the scenes identified in the play (e.g., "SCENE 1", "SCENE 2").

In [16]:
import re

#---Your code starts here---
file_path = "/content/tempest.txt"
delim = r"ACT [IVXLCDM]+\.\s*SCENE \d+"
#---Your code stops here---


tempest_text = open(file_path, "r").read()
scene_texts = re.split(delim, tempest_text)

print("Scenes:", scene_texts[1::2])

Scenes: ["\n\nOn a ship at sea; a tempestuous noise of thunder and lightning\nheard\n\nEnter a SHIPMASTER and a BOATSWAIN\n\n  MASTER. Boatswain!\n  BOATSWAIN. Here, master; what cheer?\n  MASTER. Good! Speak to th' mariners; fall to't yarely, or\n    we run ourselves aground; bestir, bestir.               Exit\n\n                       Enter MARINERS\n\n  BOATSWAIN. Heigh, my hearts! cheerly, cheerly, my hearts!\n    yare, yare! Take in the topsail. Tend to th' master's\n    whistle. Blow till thou burst thy wind, if room enough.\n\n          Enter ALONSO, SEBASTIAN, ANTONIO, FERDINAND\n                     GONZALO, and OTHERS\n\n  ALONSO. Good boatswain, have care. Where's the master?\n    Play the men.\n  BOATSWAIN. I pray now, keep below.\n  ANTONIO. Where is the master, boson?\n  BOATSWAIN. Do you not hear him? You mar our labour;\n    keep your cabins; you do assist the storm.\n  GONZALO. Nay, good, be patient.\n  BOATSWAIN. When the sea is. Hence! What cares these\n    roarers f

__3b.__ (6 points) Splitting the text on scene fails to account for act information. Update `delim` to capture the structure of both acts and scenes. Print the acts and scenes identified in the play (e.g., "ACT I. SCENE 1").

In [17]:
#---Your code starts here---
delim = r"(ACT [IVXLCDM]+\.)\s*(SCENE \d+)"
#---Your code stops here---

tempest_text = open(file_path, "r").read()
act_scene_texts = re.split(delim, tempest_text)

# Extracting and printing acts and scenes
acts_and_scenes = []
for i in range(1, len(act_scene_texts), 3):
    act = act_scene_texts[i]
    scene = act_scene_texts[i+1]
    content = act_scene_texts[i+2][:200]  # First 200 characters of the scene for brevity
    acts_and_scenes.append(f"{act} {scene}\n{content}...")

print("Acts and scenes:", act_scene_texts[1::2])

Acts and scenes: ['ACT I.', "\n\nOn a ship at sea; a tempestuous noise of thunder and lightning\nheard\n\nEnter a SHIPMASTER and a BOATSWAIN\n\n  MASTER. Boatswain!\n  BOATSWAIN. Here, master; what cheer?\n  MASTER. Good! Speak to th' mariners; fall to't yarely, or\n    we run ourselves aground; bestir, bestir.               Exit\n\n                       Enter MARINERS\n\n  BOATSWAIN. Heigh, my hearts! cheerly, cheerly, my hearts!\n    yare, yare! Take in the topsail. Tend to th' master's\n    whistle. Blow till thou burst thy wind, if room enough.\n\n          Enter ALONSO, SEBASTIAN, ANTONIO, FERDINAND\n                     GONZALO, and OTHERS\n\n  ALONSO. Good boatswain, have care. Where's the master?\n    Play the men.\n  BOATSWAIN. I pray now, keep below.\n  ANTONIO. Where is the master, boson?\n  BOATSWAIN. Do you not hear him? You mar our labour;\n    keep your cabins; you do assist the storm.\n  GONZALO. Nay, good, be patient.\n  BOATSWAIN. When the sea is. Hence! What cares t

__3c.__ (2 points) Examine the text from `ACT. I. SCENE 1`. Is `\n` a sufficient deliminator for splitting the text by speaker? Why or why not?

In [18]:
print("No, because the current output does not include the scene information correctly.")

No, because the current output does not include the scene information correctly.


__3d.__ (7 points) Complete the `process_scenes` function to separate lines in `scene_text` by `\n`, separate speakers from the beginnings of their speeches (using regex), and collect the multiline speeches in a single string. Each speaker-line pair should be stored in the `defaultdict` structure set up in the code below. Store each speaker-line pair as a list. Process `act_scene_texts` with `process_scenes`, and print the first five lines of `ACT I`, `SCENE 1`.

In [27]:
import re
from collections import defaultdict

def process_scenes(act_scene_texts):
    data = defaultdict(lambda: defaultdict(list))
    current_act, current_scene = "", ""

    # Set dictionary keys
    for i in range(1, len(act_scene_texts) - 1, 2):
        act_scene = act_scene_texts[i]
        scene_text = act_scene_texts[i + 1]

        if "ACT" in act_scene:
            current_act = re.search(r'ACT [IVXLCDM]+', act_scene).group().strip()
            current_scene = re.search(r'SCENE \d+', act_scene).group().strip() if re.search(r'SCENE \d+', act_scene) else ""
        else:
            current_scene = act_scene.strip()

        speaker, speech = "", ""

        for line in scene_text.split("\n"):
            match = re.match(r"^\s{2,}(\w+)\. (.*)", line)
            if match:
                if speaker and speech:
                    data[current_act][current_scene].append((speaker, speech.strip()))
                speaker = match.group(1)
                speech = match.group(2)
            else:
                speech += " " + line.strip()

        if speaker and speech:
            data[current_act][current_scene].append((speaker, speech.strip()))

    return data


In [28]:
data = process_scenes(act_scene_texts)
data['ACT I']['SCENE 1'][:5]

[('GONZALO',
  "Beseech you, sir, be merry; you have cause, So have we all, of joy; for our escape Is much beyond our loss. Our hint of woe Is common; every day, some sailor's wife, The masters of some merchant, and the merchant, Have just our theme of woe; but for the miracle, I mean our preservation, few in millions Can speak like us. Then wisely, good sir, weigh Our sorrow with our comfort."),
 ('ALONSO', 'Prithee, peace.'),
 ('SEBASTIAN', 'He receives comfort like cold porridge.'),
 ('ANTONIO', "The visitor will not give him o'er so."),
 ('SEBASTIAN',
  "Look, he's winding up the watch of his wit; by and by it will strike.")]

__3e.__ (5 points) Count the number of lines spoken by each speaker in the play (across every `'ACT'` and `'SCENE'`). Print the top five speakers by number of lines spoken.

In [31]:
line_counts = defaultdict(int)
for act in data:
    for scene in data[act]:
        for speaker, speech in data[act][scene]:
            line_counts[speaker] += speech.count('\n') + 1  # Count lines

# Convert line_counts to a Counter
speech_counts = Counter(line_counts)

# Print the top five speakers by number of lines spoken
top_speakers = sorted(line_counts.items(), key=lambda x: x[1], reverse=True)[:5]
print("\nTop five speakers by number of lines spoken:")
for speaker, count in top_speakers:
    print(f"{speaker}: {count} lines")

# Use speech_counts.most_common(5)
print(speech_counts.most_common(5))


Top five speakers by number of lines spoken:
SEBASTIAN: 53 lines
ANTONIO: 50 lines
GONZALO: 34 lines
STEPHANO: 29 lines
TRINCULO: 23 lines
[('SEBASTIAN', 53), ('ANTONIO', 50), ('GONZALO', 34), ('STEPHANO', 29), ('TRINCULO', 23)]


__3f.__ (2 points) Comment on any limitations of the process developed in this section. Do you see any artifacts indicating imprecision?

In [32]:
print("The developed process is limited in terms of accurately capturing multiline speeches and the completeness of the scene parsing. This limitation is evidenced by potential inaccuracies in splitting scenes and speeches, as well as possible miscounting of lines or missing speakers in some cases.")

The developed process is limited in terms of accurately capturing multiline speeches and the completeness of the scene parsing. This limitation is evidenced by potential inaccuracies in splitting scenes and speeches, as well as possible miscounting of lines or missing speakers in some cases.


# Part 4 (25 points)

In this section, you will work with a subset of the [Lending Club Loan Dataset](https://www.kaggle.com/wordsforthewise/lending-club) located in the data folder (./data/loan_extra-small.csv).

__4a.__ (3 points) Update the file path. How many fields are included in `loan_data`? How many rows are included in `load_data`?

In [1]:
import csv

#---Your code starts here---
file_path = '/content/loan_extra-small.csv'

# Initialize lists to store the fields and rows
fields = []
loan_data = []

# Open and read the CSV file
with open(file_path, mode='r', newline='', encoding='utf-8') as file:
    csvreader = csv.reader(file)

    # Extract the field names (header)
    fields = next(csvreader)

    # Extract each data row
    for row in csvreader:
        loan_data.append(row)


#---Your code stops here---

print("Number of fields: ", len(fields))
print("Number of rows:", len(loan_data))

Number of fields:  145
Number of rows: 49999


__4b.__ (2 points) Create a dictionary called `statuses` whose keys are the unique items in the `loan_status` field and whose values are set to `1` for a `loan_status` of `"Current"` or `"Fully Paid"` and `0` otherwise. Print the dictionary.

In [3]:
statuses = {}

# Populate the statuses dictionary
#---Your code starts here---
status_index = fields.index('loan_status')
for row in loan_data:
    status = row[status_index]
    if status in statuses:
        statuses[status] += 1
    else:
        statuses[status] = 1

#---Your code stops here---
# Display the statuses dictionary
statuses

{'Current': 10,
 'Fully Paid': 42429,
 'Charged Off': 7555,
 'Late (16-30 days)': 2,
 'Late (31-120 days)': 3}

__4c.__ (6 points) Count the words used in the loan descriptions (in the `desc` field) for each boolean status assigned in `statuses`. To do so, tokenize the (lowercased) description (using `word_tokenize` from `nltk.tokenize`) and then create two `Counter()` data structures that count the words in the descriptions for loans with statuses of `0` and `1`, respectively. Print the number of times the word 'loan' occurs in each set of descriptions.

In [8]:
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict

counts = defaultdict(Counter)

#---Your code starts here---
status_index = fields.index('loan_status')
desc_index = fields.index('desc')

# Define which statuses should be considered as 1 (e.g., 'Fully Paid') and which as 0
status_1_values = {'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'}
status_0_values = {'Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off'}

for row in loan_data:
    status = row[status_index]
    if status in status_1_values:
        status = 1
    elif status in status_0_values:
        status = 0
    else:
        continue  # Skip rows with statuses that are not relevant

    description = row[desc_index].lower()
    tokens = word_tokenize(description)
    counts[status].update(tokens)

#---Your code stops here---

print("Number of times 'loan' occurs in descriptions of loans with a status of 0:", counts[0]["loan"])
print("Number of times 'loan' occurs in descriptions of loans with a status of 1:", counts[1]["loan"])

Number of times 'loan' occurs in descriptions of loans with a status of 0: 3238
Number of times 'loan' occurs in descriptions of loans with a status of 1: 19423


__4d.__ (5 points) From the counters above, print the top 25 words for loans of each boolean status. How many words are in the top 25 for both status types?

In [9]:
#---Your code starts here---
top_25_status_0 = counts[0].most_common(25)
top_25_status_1 = counts[1].most_common(25)

print("Top 25 words for loans with a status of 0:")
print(top_25_status_0)

print("\nTop 25 words for loans with a status of 1:")
print(top_25_status_1)

# Find the words that are in the top 25 for both status types
words_status_0 = {word for word, _ in top_25_status_0}
words_status_1 = {word for word, _ in top_25_status_1}
common_words = words_status_0 & words_status_1

print("\nNumber of words in the top 25 for both status types:", len(common_words))
#---Your code stops here---


Top 25 words for loans with a status of 0:
[('>', 18828), ('<', 10148), ('br', 10148), ('on', 10113), ('to', 9653), ('borrower', 8731), ('added', 8698), ('i', 7300), ('and', 6039), ('my', 5813), ('credit', 5341), ('.', 4846), ('pay', 4029), ('a', 3957), ('off', 3397), ('debt', 3284), ('the', 3267), ('loan', 3238), ('cards', 2753), ('of', 2670), (',', 2633), ('have', 2432), ('for', 2428), ('this', 2311), ('card', 2042)]

Top 25 words for loans with a status of 1:
[('>', 102163), ('on', 56026), ('br', 54585), ('<', 54584), ('to', 54566), ('borrower', 47869), ('added', 47662), ('i', 41976), ('and', 32479), ('credit', 31187), ('my', 30728), ('.', 30637), ('a', 25565), ('the', 22037), ('pay', 21347), ('debt', 19745), ('loan', 19423), ('off', 19046), (',', 16056), ('of', 15125), ('cards', 15079), ('this', 14870), ('for', 13956), ('have', 13642), ('card', 13317)]

Number of words in the top 25 for both status types: 25


__4e.__ (5 points) Determine which words are unique to each status type by creating two lists: one of words that occur in descriptions of loans with statuses of `0` but not in those of loans with statuses of `1`, and one for which the opposite is true. Print the number of words in each list.

In [11]:
#---Your code starts here---
# Get all words for each status
all_words_status_0 = set(counts[0].keys())
all_words_status_1 = set(counts[1].keys())

# Determine unique words for each status
unique_to_status_0 = all_words_status_0 - all_words_status_1
unique_to_status_1 = all_words_status_1 - all_words_status_0

# Print the number of unique words for each status
print("Number of words unique to status 0:", len(unique_to_status_0))
print("Number of words unique to status 1:", len(unique_to_status_1))


Number of words unique to status 0: 1764
Number of words unique to status 1: 12773


__4f.__ (4 points) Print the top 25 unique words for loans of each boolean status. Do these sets of words indicate anything interesting about the differences between loan statuses?

In [13]:
#---Your code starts here---
# Determine unique words for each status
unique_to_status_0 = all_words_status_0 - all_words_status_1
unique_to_status_1 = all_words_status_1 - all_words_status_0

# Get the counts of the unique words
unique_counts_status_0 = {word: counts[0][word] for word in unique_to_status_0}
unique_counts_status_1 = {word: counts[1][word] for word in unique_to_status_1}

# Get the top 25 unique words for each status
top_25_unique_status_0 = Counter(unique_counts_status_0).most_common(25)
top_25_unique_status_1 = Counter(unique_counts_status_1).most_common(25)

print("Top 25 unique words for loans with a status of 0:")
print(top_25_unique_status_0)

print("\nTop 25 unique words for loans with a status of 1:")
print(top_25_unique_status_1)

# Analysis of the differences
print("\nAnalysis of the differences:")
print("The top 25 unique words for loans with a status of 0 may indicate specific reasons for default or other negative outcomes, such as terms related to financial hardship or specific payment issues.")
print("The top 25 unique words for loans with a status of 1 may reflect more positive outcomes, such as successful repayment or other favorable financial circumstances.")

#---Your code stops here---


print("\nThese descriptions show some interesting trends. The unique words for loans with a status of 0 often reflect financial struggles or negative terms that may be associated with defaulting or other repayment issues. On the other hand, the unique words for loans with a status of 1 tend to be more positive or neutral, reflecting successful repayment or financial stability.")

Top 25 unique words for loans with a status of 0:
[('ailing', 4), ('tent', 3), ('circumtance', 3), ('aarp', 3), ('moped', 3), ('payoffcreditcards', 3), ('plugged', 2), ('shirts', 2), ('exxonmobil', 2), ('doughnut', 2), ('refrigerator.', 2), ('sumer', 2), ('andother', 2), ('grades', 2), ('slavic', 2), ('enjoy.', 2), ('awaiting', 2), ('behalf.', 2), ('conslidation.', 2), ('hospitial', 2), ('payment.i', 2), ('finanacial', 2), ('unlimited', 2), ('industrial', 2), ('memorabilia', 2)]

Top 25 unique words for loans with a status of 1:
[('rsquo', 59), ('~', 44), ('rifle', 40), ('favorable', 33), ('1200', 28), ('broken', 27), ('gave', 26), ('11/16/13', 26), ('dog', 26), ('repairing', 25), ('roofing', 24), ('1,000', 23), ('750', 23), ('11/09/13', 23), ('relatively', 22), ('patrol', 22), ('atv', 21), ('liner', 21), ('defense', 21), ('daniel', 21), ('honda', 20), ('investors.', 20), ('toilet', 20), ('machine', 20), ('university', 20)]

Analysis of the differences:
The top 25 unique words for loan